# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 18.7 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO, solutions
#from google.colab import drive
#drive.mount('/content/drive')

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
# model = YOLO('/content/drive/MyDrive/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt')
model = YOLO('yolov8n.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
cap = cv2.VideoCapture('/content/highwayroad.mp4')
assert cap.isOpened(), 'The file dose not Found'

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT,  cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [ ]:
    # Calculate the center of the frame dynamically
    center_x = w // 2
    center_y = h // 2
    offset_x = 80
    offset_y = 100
    road_width = 250  # Adjust this value based on your video

    # Define the region points for the left road
    left_road_region_points = [
        [1895, 1024],[1363, 1024]
    ]

    # Define the region points for the right road
    right_road_region_points =[
[1175, 1144],[479, 1128]
]


    # Optionally, initialize your counter with these region points
    # (assuming you want to use different counters for each road)
    video_writer = cv2.VideoWriter('Video_output_miniProject.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    counter_left = solutions.ObjectCounter(view_img=False, reg_pts=left_road_region_points, names=model.names, draw_tracks=True, count_reg_color=(0, 0, 255),view_in_counts=False, line_thickness=1)
    counter_right = solutions.ObjectCounter(view_img=False, reg_pts=right_road_region_points, names=model.names, draw_tracks=True,count_reg_color=(0, 255, 0), view_in_counts=False,line_thickness=1)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.


## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print('Video frame is empty, or Video is already completed')
        break

    tracks = model.track(im0, persist=True, show=False)

    # Count vehicles in the left and right regions separately
    im0_left = counter_left.start_counting(im0, tracks)
    im0_right = counter_right.start_counting(im0, tracks)

    # Write the original frame to the output video (or potentially customized frames)
    # video_writer.write(im0)  # Or write im0_left and im0_right if needed

    im0_combined = cv2.addWeighted(im0_left, 0.5, im0_right, 0.5, 0)
    video_writer.write(im0_combined)

# Release resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 3 cars, 12.4ms
Speed: 6.1ms preprocess, 12.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 9.9ms
Speed: 3.1ms preprocess, 9.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 9.7ms
Speed: 3.2ms preprocess, 9.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 10.3ms
Speed: 3.3ms preprocess, 10.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 10.4ms
Speed: 2.9ms preprocess, 10.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 10.4ms
Speed: 3.0ms preprocess, 10.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 12.1ms
Speed: 3.2ms preprocess, 12.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 9.9ms
Speed: 3.1ms preprocess, 9.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 

KeyboardInterrupt: 

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
# Resorcess
# https://docs.ultralytics.com/guides/object-counting/#real-world-applications

## Save and Submit Your Work

Submit both the notebook and the output video